In [149]:
import tensorflow as tf
import numpy as np
import sys
from sklearn.preprocessing import normalize
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style


In [404]:
xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\new_data.csv', delimiter=',', dtype=np.float32)
x_data=xy[:,0:-1]
x_data = normalize(x_data, axis=0, norm='max')
xy[:,0:-1]=x_data
y_data=xy[:,[-1]]
m_data=xy[:499,:]
r_data=xy[:10000,:]
g_data=xy[:0,:]
F_data=xy[:,:]

In [3]:
# Model Inputs
def model_inputs(real_dim, noise_dim):
    inputs_real_ = tf.placeholder(tf.float32, shape=[None, real_dim], name='inputs_real')
    inputs_z_ = tf.placeholder(tf.float32, shape=[None, noise_dim], name='inputs_z')
    
    return inputs_real_, inputs_z_

def leaky_relu(x, alpha):
    return tf.maximum(alpha * x, x)

In [4]:
# Generator Network
def model_generator(z_input, out_dim, n_units=128, reuse=False, alpha=0.01):
    # used to reuse variables, name scope
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(z_input, n_units, activation=None)
        hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, out_dim, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [5]:
# Discriminator Network
def model_discriminator(input, n_units=128, reuse=False, alpha=0.1):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(input, n_units, activation=tf.nn.relu)
        #hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, 1, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [6]:
#parameter
input_size = 42
z_dim = 21
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.1
smooth = 0.1
learning_rate = 0.001

In [9]:
tf.reset_default_graph()  # If we don't have this, as we call this block over and over, the graph gets bigger and bigger

graph = tf.Graph()
with graph.as_default():
    inputs_real, inputs_z = model_inputs(input_size, z_dim)
    
    g_outputs, g_logits = model_generator(inputs_z, input_size, n_units=g_hidden_size, reuse=False, alpha=alpha)
    
    d_outputs_real, d_logits_real = model_discriminator(inputs_real, n_units=d_hidden_size, reuse=False, alpha=alpha)
    d_outputs_fake, d_logits_fake = model_discriminator(g_outputs, n_units=d_hidden_size, reuse=True, alpha=alpha)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real) * (1-smooth)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))
    
    d_loss = d_loss_real + d_loss_fake
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))
    
    t_vars = tf.trainable_variables()
    g_vars = [variable for variable in t_vars if 'generator' in variable.name]
    d_vars = [variable for variable in t_vars if 'discriminator' in variable.name]
    
    # Affected Variables with var_list
    d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(d_loss, var_list=d_vars)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_loss, var_list=g_vars)
    
    # Saving variables with var_list
    saver = tf.train.Saver(var_list=g_vars)

In [7]:
plt.plot(list_loss_g,list_loss_d)

NameError: name 'list_loss_g' is not defined

In [8]:
list_loss_d

NameError: name 'list_loss_d' is not defined

In [ ]:
list_loss_g

In [11]:
samples=[]
normal=0
abnormal=0
list_loss_d=[]
list_loss_g=[]
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    f = open('C:\\Users\\SANHA\\Desktop\\gen_sample100000.txt', 'a+')
    for step in range(100000):
        
        
        batch_images = FF_data[step].reshape([1, 42])
        batch_z = np.random.uniform(-1, 1, size=[1, z_dim])
        
        _ = sess.run(d_optimizer, feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        _ = sess.run(g_optimizer, feed_dict={inputs_z : batch_z})
        loss_d, loss_g = sess.run([d_loss, g_loss], feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        list_loss_d.append(loss_d)
        list_loss_g.append(loss_g)
        #if step%1000==0:
        #    print('step {} / {} Complete. D_Loss : {:0.3f}, G_Loss : {:0.3f}'.format(step+1, 100000, loss_d, loss_g))
        sys.stdout.write("\r%d / %d: %f, %f" % (step, 100000, loss_d, loss_g))
        sys.stdout.flush()
        sample_z = np.random.uniform(-1, 1, size=[1, z_dim])  # 16 Samples each epoch
        gen_samples, _ = sess.run(model_generator(inputs_z, input_size, reuse=True), feed_dict={inputs_z : sample_z})
        
        temp=gen_samples[0,41]
        #print(temp)
        if temp>=0.5:
            gen_samples[0,41]=1
            abnormal+=1
        else :
            gen_samples[0,41]=0
            normal+=1
        #print(temp,gen_samples[0,41])
        #write for text to csv
        f.write("%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f \n" %(gen_samples[0,0],gen_samples[0,1],gen_samples[0,2],gen_samples[0,3],gen_samples[0,4],gen_samples[0,5],gen_samples[0,6],gen_samples[0,7],gen_samples[0,8],gen_samples[0,9],gen_samples[0,10],gen_samples[0,11],gen_samples[0,12],gen_samples[0,13],gen_samples[0,14],gen_samples[0,15],gen_samples[0,16],gen_samples[0,17],gen_samples[0,18],gen_samples[0,19],gen_samples[0,20],gen_samples[0,21],gen_samples[0,22],gen_samples[0,23],gen_samples[0,24],gen_samples[0,25],gen_samples[0,26],gen_samples[0,27],gen_samples[0,28],gen_samples[0,29],gen_samples[0,30],gen_samples[0,31],gen_samples[0,32],gen_samples[0,33],gen_samples[0,34],gen_samples[0,35],gen_samples[0,36],gen_samples[0,37],gen_samples[0,38],gen_samples[0,39],gen_samples[0,40],gen_samples[0,41]))
       
        
    print('Generating Complete. normal={}, abnormal={}'.format(normal,abnormal))
    f.close()

28488 / 100000: 0.697456, 1.169728

KeyboardInterrupt: 

In [ ]:
xy2 = np.genfromtxt('C:\\Users\\SANHA\\Desktop\\mix_data_40000.csv', delimiter=',', dtype=np.float32)
xy3 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\gen_data.csv', delimiter=',', dtype=np.float32)
xy4 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\false_data.csv', delimiter=',', dtype=np.float32)
#gen data 20000
gx_data=xy3[:,0:-1]
gy_data=xy3[:,[-1]]
gF_data=xy3[:,:]

#gen 20000+real 20000
mx_data=xy2[:39999,0:-1]
my_data=xy2[:39999,[-1]]
mF_data=xy2[:39999,:]

#real data 40000
tx_data=r_data[:10000,0:-1]
ty_data=r_data[:10000,[-1]]

#false data
FF_data=xy4[:,:]
Fx_data=xy4[:,0:-1]
Fy_data=xy4[:,[-1]]
print("finished")

In [ ]:
nb_classes=2

X=tf.placeholder(tf.float32,[None,41])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W1=tf.Variable(tf.random_normal([41,41]),name='weight1')
b1=tf.Variable(tf.random_normal([41]),name='bias1')
layer1=tf.sigmoid(tf.matmul(X,W1)+b1)

W2=tf.Variable(tf.random_normal([41,41]),name='weight2')
b2=tf.Variable(tf.random_normal([41]),name='bias2')
layer2=tf.sigmoid(tf.matmul(layer1,W2)+b2)

W3=tf.Variable(tf.random_normal([41,nb_classes]),name='weight3')
b3=tf.Variable(tf.random_normal([nb_classes]),name='bias3')
logits=tf.matmul(layer2,W3)+b3
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

prediction=tf.argmax(hypothesis,1) #가능성을 퍼센트로~~
correct_prediction=tf.equal(prediction,tf.arg_max(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#input_F_data=input_F_data
input_train_x_data=x_data
input_train_y_data=y_data

input_test_x_data=x_data
input_test_y_data=y_data

output_x_data=x_data
output_y_data=y_data
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #f = open('C:\\Users\\SANHA\\Desktop\\false_data.txt', 'w')
    for step in range(100):
        sess.run(optimizer,feed_dict={X:input_train_x_data,Y:input_train_y_data})
        #print("training by gan sample")
        if step %10==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={X:input_test_x_data,Y:input_test_y_data})
            print("step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))


#m_data=np.append(m_data,gen_samples,axis=0)

    pred = sess.run(prediction, feed_dict={X: output_x_data})
    tr=0
    fa=0
    total=0
    fig = plt.figure(figsize=(15,9))
    ax = fig.add_subplot(1,1,1)
    ax.set(xlim=[0, 42], ylim=[0,1.5])
    for p, y in zip(pred,output_y_data.flatten()):
            if(total%100==0):
                print(total)
            if(p==int(y)): 
                tr+=1
                total+=1
                
                #if(int(y)==1):
                line, = ax.plot([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42], F_data[total,:].flatten())
                line.set(color='0.75',
                 linewidth=1, linestyle='',
                 marker='o', markersize=5, markeredgewidth=1,
                 markerfacecolor='0.75', markeredgecolor='0.75')
            else:
                fa+=1
                total+=1
                if(int(y)==1): #오탐했고 abnormal할때
                    line, = ax.plot([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42], F_data[total,:].flatten())
                    line.set(color='red',
                     linewidth=1, linestyle='',
                     marker='o', markersize=2, markeredgewidth=1,
                     markerfacecolor='red', markeredgecolor='red')
                else: #오탐했고 normal할때
                    line, = ax.plot([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42], F_data[total,:].flatten())
                    line.set(color='green',
                     linewidth=1, linestyle='',
                     marker='o', markersize=2, markeredgewidth=1,
                     markerfacecolor='green', markeredgecolor='green')
                
    line.set(color='green', label='true')
    line.set(color='red', label='false')
    plt.legend()
    plt.show()                          
   
    #f.close()
    print("true={} false: {} acc: {:0.2f}".format(tr,fa,tr/(tr+fa)))

step:     0	Loss: 1.536	Acc: 53.46%
step:    10	Loss: 0.316	Acc: 88.40%
step:    20	Loss: 0.206	Acc: 91.61%
step:    30	Loss: 0.161	Acc: 93.96%
step:    40	Loss: 0.139	Acc: 94.81%
step:    50	Loss: 0.129	Acc: 96.06%
step:    60	Loss: 0.121	Acc: 96.30%
step:    70	Loss: 0.116	Acc: 96.17%
step:    80	Loss: 0.110	Acc: 96.41%
step:    90	Loss: 0.105	Acc: 96.48%
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12